# Download the Chrome Driver to Match your Brave Version:
https://chromedriver.chromium.org/downloads  
Check your chrome version in Brave (Options => 'About Brave')  
From the Chrome Driver URL abov, download the version that matches.  
Extract the EXE to the root of this directory.

In [1]:
!pip install selenium

# Imports

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException

import os, subprocess, time

# Parameters

In [3]:
brave_install = r'C:\Program Files\BraveSoftware\Brave-Browser\Application\brave.exe'
brave_user_data_dir= r'%appdata%\Local\BraveSoftware\Brave-Browser\User Data\Default'
brave_debug_port = '9222'

# Constrect Launcher and Initalize

In [4]:
cmd = r'"{0}" --remote-debugging-port={1} --user-data-dir="{2}"'.format(
    brave_install, brave_debug_port, brave_user_data_dir)

f = open("launcher.bat", "w")
f.write(cmd)
f.close()

p = subprocess.Popen('launcher.bat', start_new_session=True)

# Connect w/ Driver

In [5]:
chrome_options = Options()
chrome_options.add_experimental_option("debuggerAddress", "localhost:9222")
driver = webdriver.Chrome(options=chrome_options)
# check connection, what is the name of the tab open?
print(f"Current Tab Open '{driver.title}'")

Current Tab Open 'New Tab'


## Test Nav

In [6]:
driver.get("http://www.google.com")
time.sleep(1)
main_window = driver.current_window_handle
print(f"Current Tab Open '{driver.title}'")

Current Tab Open 'Google'


# Test Tab Control
## Approve popups int he browser from Google.com!!!

In [9]:
driver.execute_script('''window.open("about:blank");''')

# Close all tabs except the 1st

In [10]:
if len(driver.window_handles) > 1:
    for handle in driver.window_handles[1:]:
        driver.switch_to.window(handle) 
        driver.close()
        driver.switch_to.window(driver.window_handles[0]) 